In [1]:
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
import re                                  
import string                              
from nltk.corpus import stopwords          
from nltk.stem import PorterStemmer        
from nltk.tokenize import TweetTokenizer 
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pozir\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def prepro_tweet(text):
    '''
    A function that does tokenizing, lowercasing, removing stop words and 
    punctuation and stems a string.
    '''
    stopwords_english = stopwords.words('english')
    stemmer = PorterStemmer() 

    # remove stock market tickers like $GE and old style retweet text "RT"  
    string2 = re.sub(r'\$\w*', '', text)
    string2 = re.sub(r'^RT[\s]+', '', string2)

    # remove hyperlinks
    string2 = re.sub(r'https?:\/\/.*[\r\n]*', '', string2)

    # remove # sign
    string2 = re.sub(r'#', '', string2)

    # instantiate tokenizer class
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                                   reduce_len=True)

    # tokenize tweets
    string_tokens = tokenizer.tokenize(string2)

    string_clean = []

    # remove stop words and punctuation
    for word in string_tokens:
        if (word not in stopwords_english and
            word not in string.punctuation):
            string_clean.append(word)

    # stemming the tokens
    string_stem = []

    for word in string_clean:
        stem_word = stemmer.stem(word) 
        string_stem.append(stem_word) 

    return string_stem

In [3]:
def count_dict(texts, labels):
    """count_dict.
    Input:
    texts: a list of tweets
    labels: an m x 1 array with the sentiment label of each tweet
    (either 0 or 1)
    Output:
    freqs: a dictionary mapping each (word, sentiment) pair to its
    frequency
    """
    labels = np.squeeze(labels).tolist()
    freq_dict = {}
    for tweet, y in zip(texts, labels):
        for w in prepro_tweet(tweet):
            k = (w, y)
            if k in freq_dict:
                freq_dict[k] += 1
            else:
                freq_dict[k] = 1

    return freq_dict     

In [4]:
def visual_class(labels, sizes):
    '''
    A function that visualizes the proportion of the predicted classes;
    labels: a list of the class names
    sizes: a list ofclasses counts
    '''
    # Declare a figure with a custom size
    fig = plt.figure(figsize=(5, 5))

    # Declare pie chart, where the slices will be ordered and plotted counter-clockwise:
    plt.pie(sizes, labels=labels, autopct='%1.1f%%',
            shadow=True, startangle=90)

    # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.axis('equal')
    # Display the chart
    plt.show()

In [5]:
def visual_words(list_of_words, dict_freq):
    '''
    A function that visualizes how often given words apper in positive and negative 
    tweets. The plot is in the logarithmic scale to take into account the wide 
    discrepancies between the raw counts. The red line marks the boundary between 
    positive and negative areas. Words close to the red line can be classified as 
    neutral.
    list_of_words: the list of the preprocessed words from the tweets;
    dict_freq: a dictionary of the word frequencies per each class;
    '''
    pos = 0
    neg = 0
    data = []
    
    for word in list_of_words:
        if (word, 1.0) in dict_freq:
            pos = dict_freq[(word, 1.0)]
        if (word, 0.0) in dict_freq:
            neg = dict_freq[(word, 0.0)]
        data.append([word, pos, neg])
        
    fig, ax = plt.subplots(figsize = (8, 8))

    # convert positive raw counts to logarithmic scale. we add 1 to avoid log(0)
    x = np.log([x[1] + 1 for x in data])  

    # do the same for the negative counts
    y = np.log([x[2] + 1 for x in data]) 

    # Plot a dot for each pair of words
    ax.scatter(x, y)  

    # assign axis labels
    plt.xlabel("Log Positive count")
    plt.ylabel("Log Negative count")

    # Add the word as the label at the same position as you added the points just before
    for i in range(0, len(data)):
        ax.annotate(data[i][0], (x[i], y[i]), fontsize=12)

    ax.plot([0, 9], [0, 9], color = 'red') # Plot the red line that divides the 2 areas.
    plt.show()

    return 

In [6]:
def sigmoid(z): 
    '''
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    '''      
    h = 1/(1+ 2.718281828459045**(-z))
    
    return h

In [7]:
def extract_features(tweet, freqs):
    '''
    This function takes in a single tweet, processes it using the imported prepro_tweet() function and 
    saves the list of tweet words, loop through each word in the list of processed words and for each 
    word, check the frequency dictionary for the count when that word has a positive '1' label.
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = prepro_tweet(tweet)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
    
    # loop through each word in the list of words
    for word in word_l:
        
        # increment the word count for the positive label 1
        x[0,1] += freqs.get((word, 1.0), 0)
        
        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word, 0.0), 0)
        
    assert(x.shape == (1, 3))
    return x

In [8]:
def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    '''
    m = len(x)
    iter_times = []
    costs = []
    for i in range(0, num_iters):
        
        iter_times.append(i)
        # get z, the dot product of x and theta
        z = x.dot(theta)
        
        # get the sigmoid of z
        h = sigmoid(z)
        
        # calculate the cost function
        J = (-1/m)*((y.T.dot(np.log(h)))+((1-y).T.dot(np.log(1-h))))
        #print(type(J))
        #print(J)
        costs.append(J[0][0])
        
        # update the weights theta
        theta = theta-alpha/m*(x.T.dot(h - y))
    plt.plot(iter_times, costs)
    plt.show()    
    J = float(J)
    return J, theta

In [9]:
def predict_tweet(tweet, freqs, theta):
    '''
    Given a tweet the function processes it, extracts the features, applies the model's learned weights on 
    the features to get the logits, applies the sigmoid to the logits to get the prediction
    (a value between 0 and 1). 𝑦𝑝𝑟𝑒𝑑=𝑠𝑖𝑔𝑚𝑜𝑖𝑑(𝐱⋅𝜃)
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    
    # extract the features of the tweet and store it into x
    x = extract_features(tweet, freqs)
    
    # make the prediction using x and theta
    y_pred = sigmoid(x.dot(theta))
    
    return y_pred

In [10]:
def accuracy_score(test_x, test_y, freqs, theta):
    """
    Input: 
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output: 
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """   
    # the list for storing predictions
    y_hat = []
    
    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        
        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1.0)
        else:
            # append 0 to the list
            y_hat.append(0.0)
            
    y_hat = np.array(y_hat)
    y_hat.resize((len(y_hat), 1))
    accuracy = ((y_hat == test_y).sum())/len(test_y)
    
    return accuracy

In [11]:
def single_pred(text, freqs, theta):
    print(prepro_tweet(text))
    y_hat = predict_tweet(text, freqs, theta)
    print(y_hat)
    if y_hat > 0.5:
        print('Positive sentiment')
    else:
        print('Negative sentiment')

In [12]:
def misClass(test_x, test_y, freqs, theta):
    '''
    Function that returns the tweets that were misclassified.
    '''
    print('Label Predicted Tweet')
    for x,y in zip(test_x,test_y):
        y_hat = predict_tweet(x, freqs, theta)
            
        if np.abs(y - (y_hat > 0.5)) > 0:
            print('THE TWEET IS:', x)
            print('THE PROCESSED TWEET IS:', prepro_tweet(x))
            print('%d\t%0.8f\t%s' % (y, y_hat, ' '.join(prepro_tweet(x)).encode('ascii', 'ignore')))
            print('********************************************************************************')

In [13]:
def train_naive_bayes(freqs, train_x, train_y):
    '''
    Input:
        freqs: dictionary from (word, label) to how often the word appears
        train_x: a list of tweets
        train_y: a list of labels correponding to the tweets (0,1)
    Output:
        logprior: the log prior.
        loglikelihood: the log likelihood of you Naive bayes equation. 
    '''
    N = len(train_y)
    N_pos = sum(train_y)/N
    N_neg = 1 - N_pos
    logprior = np.log(N_pos) - np.log(N_neg)
    # create the vocabulary of the unique words
    vocab = set([i[0] for i in freqs.keys()])
    V = len(vocab)
    # the number of words in the vocabulary per class
    num_words_pos  = 0
    num_words_neg = 0
    
    for w in vocab:
        
        num_words_pos += freqs.get((w, 1), 0)
        num_words_neg += freqs.get((w, 0), 0)
    
    loglikelihood = {}
    
    for w in vocab:
        
        aux_pos = (freqs.get((w, 1), 0) + 1)/(num_words_pos + V)
        aux_neg = (freqs.get((w, 0), 0) + 1)/(num_words_neg + V)
        loglikelihood[w] = np.log(aux_pos/aux_neg)
        
    return logprior, loglikelihood 

'C:\\Sentiment_Analysis_LogReg'

In [ ]:
def naive_bayes_predict(text, logprior, loglikelihood):
    '''
    Input:
        tweet: a string
        logprior: a number
        loglikelihood: a dictionary of words mapping to numbers
    Output:
        p: the sum of all the logliklihoods of each word in the tweet (if found in the dictionary) + logprior (a number)

    '''
    tok_list = prepro_tweet(text)
    log_like_sum = 0
    for i in tok_list:
        log_like_sum += loglikelihood.get(i, 0)
    return logprior + log_like_sum  

In [ ]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    """
    Input:
        test_x: A list of tweets
        test_y: the corresponding labels for the list of tweets
        logprior: the logprior
        loglikelihood: a dictionary with the loglikelihoods for each word
    Output:
        accuracy: (# of tweets classified correctly)/(total # of tweets)
    """
    
    predictions = []
    for i in test_x:
        score = naive_bayes_predict(i, logprior, loglikelihood)
        if score > 0:
            predictions.append(1)
        else:
            predictions.append(0)
    
    predictions = np.array(predictions)
    accuracy_score = 1 - (sum(abs(predictions-test_y)))/(len(test_y))
    
    return accuracy_score    

In [ ]:
def get_ratio(freqs, word):
    '''
    Input:
        freqs: dictionary containing the words
        word: string to lookup

    Output: a dictionary with keys 'positive', 'negative', and 'ratio'.
        Example: {'positive': 10, 'negative': 20, 'ratio': 0.5}
    '''
    pos_neg_ratio = {'positive': 0, 'negative': 0, 'ratio': 0.0}
    pos_neg_ratio['positive'] = freqs.get((word, 1), 0)
    pos_neg_ratio['negative'] = freqs.get((word, 0), 1)
    pos_neg_ratio['ratio'] = (pos_neg_ratio['positive'] + 1)/(pos_neg_ratio['negative'] + 1)
    
    
    return pos_neg_ratio

In [ ]:
def get_words_by_threshold(freqs, label, threshold):
    '''
    Input:
        freqs: dictionary of words
        label: 1 for positive, 0 for negative
        threshold: ratio that will be used as the cutoff for including a word in the returned dictionary
    Output:
        word_set: dictionary containing the word and information on its positive count, negative count, and ratio of positive to negative counts.
        example of a key value pair:
        {'happi':
            {'positive': 10, 'negative': 20, 'ratio': 0.5}
        }
    '''
    word_list = {}

    for key in freqs.keys():
        word, _ = key

        # get the positive/negative ratio for a word
        pos_neg_ratio = get_ratio(freqs, word)['ratio']

        # if the label is 1 and the ratio is greater than or equal to the threshold...
        if label == 1 and pos_neg_ratio >= threshold :

            # Add the pos_neg_ratio to the dictionary
            word_list[word] = {'positive':get_ratio(freqs, word)['positive'] , 
                               'negative': get_ratio(freqs, word)['negative'],
                               'ratio': pos_neg_ratio}

        # If the label is 0 and the pos_neg_ratio is less than or equal to the threshold...
        elif label == 0 and pos_neg_ratio <= threshold:

            # Add the pos_neg_ratio to the dictionary
            word_list[word] = {'positive':get_ratio(freqs, word)['positive'] , 
                               'negative': get_ratio(freqs, word)['negative'],
                               'ratio': pos_neg_ratio}

    return word_list

In [ ]:
def missclassNb(test_x, test_y, logprior, loglikelihood):
    print('Truth Predicted Tweet')
    for x, y in zip(test_x, test_y):
        y_hat = naive_bayes_predict(x, logprior, loglikelihood)
        if y != (np.sign(y_hat) > 0):
            print('%d\t%0.2f\t%s' % (y, np.sign(y_hat) > 0, ' '.join(
                prepro_tweet(x)).encode('ascii', 'ignore')))

In [ ]:
def tweet_likelihood(freqs, train_y, train_x):
    '''
    Input:
        freqs: dictionary from (word, label) to how often the word appears
        train_x: a list of tweets
        train_y: a list of labels correponding to the tweets (0,1)
    Output:
        logprior: the log prior.
        loglikelihood: the log likelihood of you Naive bayes equation. 
    '''
    
    # create the vocabulary of the unique words
    vocab = set([i[0] for i in freqs.keys()])
    V = len(vocab)
    # the number of words in the vocabulary per class
    num_words_pos  = 0
    num_words_neg = 0
    
    for w in vocab:
        
        num_words_pos += freqs.get((w, 1), 0)
        num_words_neg += freqs.get((w, 0), 0)
    
    loglikelihood = {}
    
    for w in vocab:
        
        aux_pos = (freqs.get((w, 1), 0) + 1)/(num_words_pos + V)
        aux_neg = (freqs.get((w, 0), 0) + 1)/(num_words_neg + V)
        loglikelihood[(w, 1)] = aux_pos
        loglikelihood[(w, 0)] = aux_neg
    
    pos = []
    neg = []
    for i in train_x:
        
        token_list = prepro_tweet(i)
        positive = sum([np.log(loglikelihood[(t, 1)]) for t in token_list])
        pos.append(positive)
        negative = sum([np.log(loglikelihood[(t, 0)]) for t in token_list])
        neg.append(negative)

    d = {'positive': pos, 'negative': neg, 'sentiment': list(train_y)}
    bayes_feat = pd.DataFrame(d)
    
        
    return bayes_feat

In [ ]:
def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of *x* and *y*.

    Parameters
    ----------
    x, y : array-like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    **kwargs
        Forwarded to `~matplotlib.patches.Ellipse`

    Returns
    -------
    matplotlib.patches.Ellipse
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=facecolor, **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)